<a href="https://colab.research.google.com/github/atharvgarg/financial_news_summary/blob/main/Optimal_Summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare Environment

In [1]:
!pip install transformers datasets sentencepiece
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 6.8 MB/s 
     |████████████████████████████████| 365 kB 65.2 MB/s 
     |████████████████████████████████| 1.3 MB 65.7 MB/s 
     |████████████████████████████████| 120 kB 70.4 MB/s 
     |████████████████████████████████| 6.6 MB 47.2 MB/s 
     |████████████████████████████████| 115 kB 68.6 MB/s 
     |████████████████████████████████| 212 kB 70.1 MB/s 
     |████████████████████████████████| 127 kB 73.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!git clone https://github.com/nadasalah99/bert-extractive-summarizer

Cloning into 'bert-extractive-summarizer'...
remote: Enumerating objects: 276, done.
remote: Total 276 (delta 0), reused 0 (delta 0), pack-reused 276
Receiving objects: 100% (276/276), 57.15 KiB | 4.40 MiB/s, done.
Resolving deltas: 100% (157/157), done.


In [3]:
!pip install -r /content/bert-extractive-summarizer/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 17.3 MB 6.2 MB/s 
     |████████████████████████████████| 753.4 MB 6.9 kB/s 
     |████████████████████████████████| 10.6 MB 50.6 MB/s 
     |████████████████████████████████| 387 kB 73.6 MB/s 
     |████████████████████████████████| 2.1 MB 52.0 MB/s 
     |████████████████████████████████| 50 kB 8.4 MB/s 
     |████████████████████████████████| 286 kB 70.1 MB/s 
     |████████████████████████████████| 7.1 MB 57.8 MB/s 
     |████████████████████████████████| 248 kB 57.3 MB/s 
     |████████████████████████████████| 576 kB 69.8 MB/s 
     |████████████████████████████████| 880 kB 65.4 MB/s 
     |████████████████████████████████| 132 kB 67.7 MB/s 
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of neuralcoref to det

In [4]:
!pip install tqdm
!pip install spacy~=2.2.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached spacy-2.2.4-cp37-cp37m-manylinux1_x86_64.whl (10.6 MB)
     |████████████████████████████████| 3.7 MB 8.0 MB/s 
     |████████████████████████████████| 184 kB 66.2 MB/s 
     |████████████████████████████████| 2.2 MB 57.1 MB/s 
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.4
    Uninstalling srsly-2.4.4:
      Successfully uninstalled srsly-2.4.4
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.8
    Uninstalling catalogue-2.0.8:
      Successfully uninstalled catalogue-2.0.8
  Attempting uninstall: blis
    Found existing installation: blis 0.7.8
    Uninstalling blis-0.7.8:
      Successfully uninstalled blis-0.7.8
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.0
    Uninstalling thinc-8.1.0

# Import libraries

In [5]:
import pandas as pd
import numpy as np
import pickle
import re
import os
import string
import textwrap
import heapq

from transformers import AutoTokenizer, AutoModel,  AutoModelForSeq2SeqLM, pipeline

import torch

import logging
from sklearn.model_selection import train_test_split
from sklearn import preprocessing,metrics


import nltk
from nltk import tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('punkt')
st = PorterStemmer()
nltk.download('stopwords')
stop=stopwords.words('english')

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Text Classification

In [6]:
def clean(text):
  #remove hashtags
  text = re.sub("#", " ", text)
  #remove all numbers 
  #text = re.sub(r'\[0-9]*\]',' ',text)
  #remove duplicated chars
  #text = re.sub(r'(.)\1+', r'\1', text)
  #remove :) or :(
  text = text.replace(':)', "")
  text = text.replace(':(', "")
  #remove multiple exclamation
  text = re.sub(r"(\!)\1+", ' ', text)
  #remove multiple question marks
  text = re.sub(r"(\?)\1+", ' ', text)
  #remove multistop
  text = re.sub(r"(\.)\1+", ' ', text)
  #remove additional spaces
  text = re.sub(r"[\s]+", " ", text)
  text = re.sub(r"[\n]+", " ", text)
  
  return text

def remStopWords(Text):
  return " ".join(word for word in Text.split() if word not in stop)

def stemWords(Text):
  return " ".join(st.stem(word) for word in Text.split())
  
def pipeline(Text):
  #preprocessing step
  Text = clean(Text)
  Text = "".join([char for char in Text if char not in string.ascii_letters]).strip()
  Text = remStopWords(str(Text))
  Text = stemWords(Text)

# Extractive Model

In [7]:
cd /content/bert-extractive-summarizer

/content/bert-extractive-summarizer


In [8]:
ls

arabert_summarize_example.py  Makefile                  setup.cfg
Dockerfile                    README.md                 setup.py
Dockerfile.service            requirements-service.txt  summarize.py
example_file.txt              requirements.txt          summarizer/
LICENSE                       server.py                 tests/


In [9]:
from summarizer import Summarizer 

extractive_Model = AutoModel.from_pretrained('bert-base-uncased')
extractive_Tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

extractiveModelSummarizer = Summarizer(custom_model=extractive_Model, custom_tokenizer=extractive_Tokenizer)

https://huggingface.co/bert-base-uncased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpogusppur


storing https://huggingface.co/bert-base-uncased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
creating metadata file for /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "h

storing https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f
creating metadata file for /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f
loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.we

storing https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
creating metadata file for /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob":

storing https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
creating metadata file for /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpizggq7k_


storing https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
creating metadata file for /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://hugg

## Model to Pickle

In [11]:
pickle.dump(extractiveModelSummarizer, open("/content/Data/Models/extractiveModel.pkl", "wb"))  

# Abstractive Model

In [12]:
model_name = "Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old"
abstractive_Tokenizer = AutoTokenizer.from_pretrained(model_name)
abstractive_Model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpo2ba69sn


storing https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/4aef3fc730005e8f1645715e0bf48f660dea26701ac359129d876e3233789467.d24951e82532aeeaee7ab02281590ea82c08b4dcc773db0c892dee877cd533cd
creating metadata file for /root/.cache/huggingface/transformers/4aef3fc730005e8f1645715e0bf48f660dea26701ac359129d876e3233789467.d24951e82532aeeaee7ab02281590ea82c08b4dcc773db0c892dee877cd533cd
https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpk_wh9ydx


storing https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/cabd85f7cba6b92e9fb7b2406e0b8da2da2a746bdcf1c980e1b59e984786c565.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
creating metadata file for /root/.cache/huggingface/transformers/cabd85f7cba6b92e9fb7b2406e0b8da2da2a746bdcf1c980e1b59e984786c565.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpdd22h8ok


storing https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/c61417a68695519ccbdcf9599b3377624fefc4633bb964f3289cf4b4593a4a38.c32c4a7596b5bfd7275f3afee64968a94b9951235630e31c699108573b683baf
creating metadata file for /root/.cache/huggingface/transformers/c61417a68695519ccbdcf9599b3377624fefc4633bb964f3289cf4b4593a4a38.c32c4a7596b5bfd7275f3afee64968a94b9951235630e31c699108573b683baf
https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpn5lcrld1


storing https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/59b9c5d5f0c8316c2b56431c1c78c97caf104f2911b2628445c5a15540888cf0.7da70648c6cb9951e284c9685f9ba7ae083dd59ed1d6d84bdfc0584a4ea94b6d
creating metadata file for /root/.cache/huggingface/transformers/59b9c5d5f0c8316c2b56431c1c78c97caf104f2911b2628445c5a15540888cf0.7da70648c6cb9951e284c9685f9ba7ae083dd59ed1d6d84bdfc0584a4ea94b6d
loading file https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/cabd85f7cba6b92e9fb7b2406e0b8da2da2a746bdcf1c980e1b59e984786c565.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news

storing https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/c147149d80d9bf9f3471c6b017caef96326ba1fa4359c1cd7433fcfc4fc9eb94.320d20ca4053e17fa9777800a62434d5f9654b9adfea417539328eca86250159
creating metadata file for /root/.cache/huggingface/transformers/c147149d80d9bf9f3471c6b017caef96326ba1fa4359c1cd7433fcfc4fc9eb94.320d20ca4053e17fa9777800a62434d5f9654b9adfea417539328eca86250159
loading configuration file https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c147149d80d9bf9f3471c6b017caef96326ba1fa4359c1cd7433fcfc4fc9eb94.320d20ca4053e17fa9777800a62434d5f9654b9adfea417539328eca86250159
Model config EncoderDecoderConfig {
  "_name_or_path": "Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summariz

storing https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/9b0b934276ba3160f6c5f2302d5abc8519738b8688a3e2ddac6d695bb8be6043.fafdc0774d398784fa45105017064d8b4c5fd020b63bc1fc0251de8108826e53
creating metadata file for /root/.cache/huggingface/transformers/9b0b934276ba3160f6c5f2302d5abc8519738b8688a3e2ddac6d695bb8be6043.fafdc0774d398784fa45105017064d8b4c5fd020b63bc1fc0251de8108826e53
loading weights file https://huggingface.co/Atharvgarg/bert-small2bert-small-finetuned-cnn_daily_mail-summarization-finetuned-bbc-news-old/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9b0b934276ba3160f6c5f2302d5abc8519738b8688a3e2ddac6d695bb8be6043.fafdc0774d398784fa45105017064d8b4c5fd020b63bc1fc0251de8108826e53
All model checkpoint weights were used when initializing EncoderDecoderModel.

All the weights of EncoderDecoderMode

In [13]:
def abstractiveModelSummarizer(text):
  input_ids = abstractive_Tokenizer(
      [(text)],
      return_tensors="pt",
      padding="max_length",
      truncation=True,
      max_length=512
  )["input_ids"]

  output_ids = abstractive_Model.generate(
      input_ids=input_ids,
      max_length=200,
      no_repeat_ngram_size=8,
      num_beams=4
  )[0]

  summary = abstractive_Tokenizer.decode(
      output_ids,
      skip_special_tokens=True,
      clean_up_tokenization_spaces=False
  )
  return summary

## Model to Pickle

In [14]:
pickle.dump(abstractiveModelSummarizer, open("/content/Data/Models/abstractiveModelbbc.pkl", "wb"))  

## Model to Pickle

#Hybrid Summary


In [15]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=f9be23aeed56a25af9938c19cfc69dfb23b8427c06d4fd84e0bcc992e2708187
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [16]:
from sentence_transformers import SentenceTransformer, util
from nltk.tokenize.treebank import TreebankWordDetokenizer


hybridModel = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

loading configuration file /root/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/config.json
Model config BertConfig {
  "_name_or_path": "/root/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /root/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/pytorch_model.bin
All model checkpoint weights were used when i

In [19]:
def hybrid_summary(abs_sum, ext_sum):
    s1 = tokenize.sent_tokenize(clean(abs_sum))
    s2 = tokenize.sent_tokenize(clean(ext_sum))

    if len(s1) == len(s2):
        pass
    else:
        extra_sentences = len(s1)-len(s2)
        s2.extend(s1[:extra_sentences])

    #Compute embedding for both lists
    embedding_1= hybridModel.encode(s1, convert_to_tensor=True)
    embedding_2 = hybridModel.encode(s2, convert_to_tensor=True)

    cosine_scores = util.pytorch_cos_sim(embedding_1, embedding_2)


    sents = []
    for i in range(len(s1)):
        sents.append(
        {
            'Abstractive': s1[i],
            'Extractive': s2[i],
            'Similarity':  cosine_scores.tolist()[i][i]
        })
    merge_threshold = 0.15
    lower_threshold = 0.09
    hybrid_sum = []
    for i in range(len(sents)):
        if sents[i]['Similarity'] > merge_threshold:
            hybrid_sum.append(s1[i])
        elif sents[i]['Similarity'] < lower_threshold:
            pass
        else:
            hybrid_sum.extend([s1[i],s2[i]])

    return TreebankWordDetokenizer().detokenize(hybrid_sum)

In [28]:
pickle.dump(hybrid_summary, open("/content/Data/Models/similarity_model.pkl", "wb"))  

# Load Pickles

In [29]:
extractiveModel = pickle.load(open('/content/Data/Models/extractiveModel.pkl', 'rb'))

abstractiveModel = pickle.load(open('/content/Data/Models/abstractiveModelbbc.pkl', 'rb'))

hybrid_summary = pickle.load(open('/content/Data/Models/similarity_model.pkl', 'rb'))  

# The Models Output

In [24]:
article = """Stranger Things is an American science fiction drama television 
series created by the Duffer Brothers, who also serve as showrunners and are
executive producers along with Shawn Levy and Dan Cohen. Its first season
was released on Netflix on July 15, 2016. In February 2022, the series was 
renewed for a fifth and final season.

Set in the 1980s, primarily in the fictional town of Hawkins, Indiana, the 
series centers on a number of mysteries and supernatural events occurring
around the town and their impact on an ensemble of child and adult characters.
It stars Winona Ryder, David Harbour, Finn Wolfhard, Millie Bobby Brown, 
Gaten Matarazzo, Caleb McLaughlin, Natalia Dyer, Charlie Heaton, Cara Buono,
Matthew Modine, Noah Schnapp, Sadie Sink, Joe Keery, Dacre Montgomery, 
Sean Astin, Paul Reiser, Maya Hawke, Priah Ferguson, and Brett Gelman.

The Duffer Brothers developed Stranger Things as a mix of investigative drama
and supernatural elements portrayed with horror and childlike sensibilities,
while infusing references to the pop culture of the 1980s. Several thematic 
and directorial elements were inspired by the works of Steven Spielberg, 
John Carpenter, David Lynch, Stephen King, Wes Craven and H. P. Lovecraft. 
They also took inspiration from experiments conducted during the Cold War 
and conspiracy theories involving secret government experiments.

One of Netflix's flagship series, Stranger Things has attracted record 
viewership on the streaming platform. The series has been critically 
acclaimed for its characterization, atmosphere, acting, soundtrack, directing,
writing, and homages to 1980s films. It has received numerous awards and
nominations, including nominations from the Golden Globe Awards, British 
Academy Television Awards, Directors Guild of America Awards, Writers 
Guild of America Awards, and Grammy Awards, as well as wins from the
Primetime Emmy Awards, Screen Actors Guild Awards, American Film Institute,
Critics' Choice Television Awards, and People's Choice Awards."""

In [25]:
the_result_extractive = extractiveModel(article)
print(textwrap.fill(the_result_extractive,100))

Stranger Things is an American science fiction drama television  series created by the Duffer
Brothers, who also serve as showrunners and are executive producers along with Shawn Levy and Dan
Cohen. Its first season was released on Netflix on July 15, 2016. The Duffer Brothers developed
Stranger Things as a mix of investigative drama and supernatural elements portrayed with horror and
childlike sensibilities, while infusing references to the pop culture of the 1980s.


In [26]:
the_result_abstractive = abstractiveModel(article)  
print(textwrap.fill(the_result_abstractive,100))

the duffer brothers developed stranger things as a mix of investigative drama and supernatural
elements portrayed with horror and childlike sensibilities, while infusing references to the pop
culture of the 1980s. one of netflix's flagship series, stranger things has attracted record
viewership on the streaming platform. it has received numerous awards and nominations, including
nominations from the golden globe awards, british academy television awards, directors guild of
america awards, and grammy awards, as well as wins from the primetime emmy awards, writers guild of
america awards, and people's choice awards, and people's choice award. the duffer brothers developed
stranger things is an american science fiction drama television series created by the duffer
brothers, who also serve as showrunners and are executive producers along with shawn levy and dan
cohen. set in the 1980s, primarily in the fictional town of hawkins, indiana, the series centers on
a number of mysteries and supe

In [30]:
the_result_hybrid = hybrid_summary(the_result_abstractive,the_result_extractive)
print(textwrap.fill(the_result_hybrid,100))

the duffer brothers developed stranger things as a mix of investigative drama and supernatural
elements portrayed with horror and childlike sensibilities, while infusing references to the pop
culture of the 1980s. one of netflix's flagship series, stranger things has attracted record
viewership on the streaming platform. the duffer brothers developed stranger things is an american
science fiction drama television series created by the duffer brothers, who also serve as
showrunners and are executive producers along with shawn levy and dan cohen. set in the 1980s,
primarily in the fictional town of hawkins, indiana, the series centers on a number of mysteries and
supernatural events occurring around the town and their impact on an ensemble of
